<a href="https://colab.research.google.com/github/katrin2202/ML-in-forecasting-tasks/blob/main/Lecture_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключаем нужные библиотеки

In [1]:
import pandas as pd
import io
import requests

Подтягиваем данные по таблице 2.1 Год вступления в организацию и текущий статус членства.

*read_csv* - чтение файла csv. передаем ссылку на файл

In [3]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")

Подтягиваем данные по таблице 2.2 Недельное количество писам, отправленных члену организации, которые были им открыты

In [4]:
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")

Подтягиваем данные по таблице 2.3 Данные о добровольных взносах

In [5]:
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")

Проверяем, есть ли у членов организации более одной записи. Видим что все 1000 членов имеют только 1 статус

*groupby()* - используется для разделения и выделения некоторой части данных из целого набора данных на основе определенных предопределенных условий или параметров.

*count()* - количество

In [7]:
# 55 page
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


проверям на пустые от писем недели. видим что их таких нет, значит в эти недели письма не прходили

In [8]:
# 56 page
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


Посмотрим работу с почтой пользователя 998 

In [9]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


Заметим что не все недели года учтены. Посчитаем количество еженедельных наблюдений между первым и последним событием.

In [10]:
#57 page
(pd.to_datetime(max(emails[emails.user== 998].week))-pd.to_datetime(min(emails[emails.user==998].week))).days/7

25.0

альтернативная реализация кода выше с использованием datetime

In [11]:
# 57 page
import datetime
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

а теперь посмотрим сколько недель зарегистрированных за работай

In [12]:
emails[emails.user == 998].shape

(24, 3)

видим несоответствие получается всего 998 пользователь был 26 недель, но письма открывал только 24 недели.

заполним недостающие недели с помощью Multiindex в Pandas. Для этого получим все комбинации недель и членов (декартово произведение). и теперь можем заполнить все недостающие нам недели

In [13]:
 # 58
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))
complete_idx

MultiIndex([('2015-11-02 00:00:00',   1.0),
            ('2015-11-02 00:00:00',   3.0),
            ('2015-11-02 00:00:00',   5.0),
            ('2015-11-02 00:00:00',   6.0),
            ('2015-11-02 00:00:00',   9.0),
            ('2015-11-02 00:00:00',  10.0),
            ('2015-11-02 00:00:00',  14.0),
            ('2015-11-02 00:00:00',  16.0),
            ('2015-11-02 00:00:00',  20.0),
            ('2015-11-02 00:00:00',  21.0),
            ...
            ('2017-05-15 00:00:00', 973.0),
            ('2017-05-15 00:00:00', 977.0),
            ('2017-05-15 00:00:00', 982.0),
            ('2017-05-15 00:00:00', 984.0),
            ('2017-05-15 00:00:00', 987.0),
            ('2017-05-15 00:00:00', 991.0),
            ('2017-05-15 00:00:00', 992.0),
            ('2017-05-15 00:00:00', 993.0),
            ('2017-05-15 00:00:00', 995.0),
            ('2017-05-15 00:00:00', 998.0)],
           length=93247)

сбрасываем индекс, чтобы заносить информацию членах организаци и неделях отдельные столбцы, затем присваиваем имена этим столбцам.

In [14]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

Проверям

In [15]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
66296,2015-02-09 00:00:00,998.0,0.0
58211,2015-02-16 00:00:00,998.0,0.0
71147,2015-02-23 00:00:00,998.0,0.0
14013,2015-03-02 00:00:00,998.0,0.0
49587,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
31261,2018-04-30 00:00:00,998.0,3.0
3233,2018-05-07 00:00:00,998.0,3.0
25871,2018-05-14 00:00:00,998.0,3.0
63601,2018-05-21 00:00:00,998.0,3.0


соберем таблицу где будет видны первая и последняя неделя прочитанного писем

In [16]:
# 59 page
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()
cutoff_dates

,index,user,min,max
0,0,1.0,2015-06-29 00:00:00,2018-05-28 00:00:00
1,1,3.0,2018-03-05 00:00:00,2018-04-23 00:00:00
2,2,5.0,2017-06-05 00:00:00,2018-05-28 00:00:00
3,3,6.0,2016-12-05 00:00:00,2018-05-28 00:00:00
4,4,9.0,2016-07-18 00:00:00,2018-05-28 00:00:00
...,...,...,...,...
534,534,991.0,2016-10-24 00:00:00,2016-10-24 00:00:00
535,535,992.0,2015-02-09 00:00:00,2015-07-06 00:00:00
536,536,993.0,2017-09-11 00:00:00,2018-05-28 00:00:00
537,537,995.0,2016-09-05 00:00:00,2018-05-28 00:00:00


удалим для каждого пользователя все нулевые недели до первого и после последнего не нулевого значения

In [17]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

Перестроим данные о взносах, превратим и их в недельные временные ряды.

In [18]:
# 61 page
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

Посмотрим что получилось

In [19]:
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

Создадим сводную таблицу объединив таблицу с письмами и взновами по пользователю и неделе.

In [20]:
#61 page
merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
  user_donations = agg_don[agg_don.index.get_level_values('user') == user]
  
  user_donations = user_donations.droplevel(0)
  #user_donations.set_index('timestamp', inplace = True)  
  user_email.set_index('week', inplace=True)

  user_email = all_email[all_email.user == user]
  user_email.sort_values('week', inplace=True)

  df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
  df.fillna(0)

  merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

выведем начала общей таблицы

In [21]:
merged_df.head()

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN


мы будем смотреть зависимость взносов в следующей недели от тикущих открытых писем. для этого в новой столбце сдвинем все взносы на одну строку.

In [22]:
#62 page
df = merged_df[merged_df.user == 998]
df['target'] = df.amount.shift(1)
df = df.fillna(0)
df.head(10)

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04 00:00:00,1.0,0.0,0.0
1,998.0,2017-12-11 00:00:00,3.0,0.0,0.0
2,998.0,2017-12-18 00:00:00,3.0,0.0,0.0
3,998.0,2017-12-25 00:00:00,0.0,0.0,0.0
4,998.0,2018-01-01 00:00:00,3.0,0.0,0.0
5,998.0,2018-01-08 00:00:00,3.0,0.0,0.0
6,998.0,2018-01-15 00:00:00,2.0,0.0,0.0
7,998.0,2018-01-22 00:00:00,3.0,0.0,0.0
8,998.0,2018-01-29 00:00:00,2.0,0.0,0.0
9,998.0,2018-02-05 00:00:00,3.0,0.0,0.0


In [23]:
# page 89
air = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/AirPassengers.csv",
                  names=['Date','Passengers'])

In [24]:
air

,Date,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


Для сглаживания данных о количестве перевозимых пассажиров применяется функция ewma() пакета Pandas, в которой учитываются разные парметры затухания

###Python

In [25]:
#89 page
air['Smooth.5'] = air.ewm(alpha = .5).Passengers.mean()
air['Smooth.9'] = air.ewm(alpha = .9).Passengers.mean()

Легко заметить, что значение параметра ***alpha***, определяющего ***коэффициент сглаживания***, показывает, насколько сильно изменилось значение до его текущей величины по сравнени. с вычисленным средним значением. Чем больше значение ***alpha***, тем сильнее приближение к текущей величине. Заметьте, что функции сглаживания библиотеки ***Pandas*** принимают несколько параметров, подставляемых в одну и ту же расчетную формулу, но задающих несколько способов интерпретации экспоненциального скользящего среднего(1).


---
(1)В реляционных баззах данных информация традиционно хранится в таблицах. Например, для представления сведений о транзакциях клиентов в такую базу данных обычно добавляются таблица с полной информацией о клиентах, включая уникальный идентификатор, и таблица с данными транзакций, в которой одно из полей отводится для хранения идентификаторов клиентов.

In [26]:
air

,Date,Passengers,Smooth.5,Smooth.9
0,1949-01,112,112.000000,112.000000
1,1949-02,118,116.000000,117.454545
2,1949-03,132,125.142857,130.558559
3,1949-04,129,127.200000,129.155716
4,1949-05,121,124.000000,121.815498
...,...,...,...,...
139,1960-08,606,582.096411,606.665454
140,1960-09,508,545.048205,517.866545
141,1960-10,461,503.024103,466.686655
142,1960-11,390,446.512051,397.668665


Однако в случе данных с долгосрочным трендом простое экспоненциальное сглаживание не позволяет добиться хороших результатов в прогнозировании. Поэтому сглаживание данных с ярко выраженным трендом и с сезонностью, выполняется с помощью двух специальных экспоненциальных методов - Холта и Холта-Уинтерса.

Часовые пояса усложняют обработку временных данных, а с появлением персональных компьютеров таких сложностей стало несравнимо больше:

*   часовые пояса определяются социоло-политически
*   отсутсвует единый способ передачи информации о часовом поясе между разными языками програмирования или по протоколу HTTP
*   отсутсвие единого протокола именований или определения даты начала и окончания перехода на летнее время в часовых поясах
*   переход на летнее время в некоторых часовых поясах иногда происходит дважды в год

Большинство языков программирования получают информацию о часовом поясе от операционной системы. К сожалению, встроенная функция автоматического определения даты и времени в языке Python, **datetime.now()**, не возвращает врменную метку, учитывающую часовой пояс. Отчасти это сделано преднамеренно. Некоторые решения, принятые в стандартной библиотеке, поддерживают запрет на обработку информации о часовом поясе, относящейся к модулю datetime (из-за частых изменений в нем), и обьекты datetime как с включенной информацией о часовом поясе, так и без нее. При этом сравнение обьектов времени с данными часового пояса и без них выдаст ошибку TypeError.

 Некоторые исследователи утверждают, что большинство программных библиотек Python написаны в предложении, что **tzinfo == None**. Несмотря на недоказуемость, скорее всего, это действительно так. Многие также сообщают о сложностях сериализации обьектов, снабженных временными метками с указанием часового пояса, поэтому, если вы планируете подвергать ей свои данные, заранее удостоверьтесь в справедливости или невозможности выполнения этой оперции(смотри подробнее:https://perma.cc/4265-4u8l)
 
 Давайте посмотрим, как обрабатываются данные часовых поясов в языке Python. Чаще всего для этой цели применяются инструменты библиотек **datetime**, **pytz** и **dateutil**. Кроме того, многие удобные функции управления обьектами с данными о часовых поясах содержатся в библиотек **Pandas** - они основаны на средствах последних двух библиотек.

 Ниже рассмотрены только наиболее выжные функции управления данными часовых поясов.
 
 Во-первых, функция **now()** модуля **datetime** не предоставляет информацию о часовом поясе, хотя и учитывает его при определении (правильного) времени + различие в данных в возвращаемых функциями **now()** и **utcnow()**





In [27]:
# 96 page
import datetime
datetime.datetime.utcnow()

datetime.datetime(2022, 9, 18, 6, 56, 34, 851169)

In [28]:
datetime.datetime.now()

datetime.datetime(2022, 9, 18, 6, 56, 35, 431183)

In [29]:
datetime.datetime.now(datetime.timezone.utc)

datetime.datetime(2022, 9, 18, 6, 56, 36, 170469, tzinfo=datetime.timezone.utc)

Чтобы получить правильны резельтат, достаточно перейти к формату с указанием часового пояса, но это нестандартное поведение, Для управления часовыми поясами в языке Python создается обьект часового пояса, например **western**, представляющий тихоокеанский часовой пояс США

In [30]:
#97 page
import pytz

western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

Впоследствии такие обьекты применяются для преобразования часового пояса в данные с помощью функиции **localize()**.

тут нужно вставить код или это пояснение кода, это уж тебе решать:
 
## Python
> API предоставляет два способа вычисления времени с учетом 
> часового пояса - ***'локализацию'*** и приведение времени к формату
> другого часового пояса.
> В нашем случае выполняется ***'локализацию'***

In [31]:
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
loc_dt

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

Однако не забываейте, что передача часового пояса непосредственно в конструктор **datetime()** часто приводит к совершенно неожиданному результату.

In [32]:
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [33]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)

'2018-05-12 12:15:00 LMT-0001'

> Как подчеркивается в документации к модулю *pytz*,
> задание параметра *tzinfo* в функции инициализации
> datetime.datetime не всегда приводит к получению желаемого
> результата, как в случае г. Лондона.
> Согласно документации к библиотеке *pytz*, метод работает только
> в данных с часовыми поясами без перехода на летнее время

## Python
> Как правило, данные времени хранятся в формате UTC
> и преобразовываются в другие форматы только при выводе
> для изучения людьми. Кроме того, к часовым поясам часто
> применяются арифметические операции

In [34]:
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1

datetime.timedelta(days=1, seconds=17520)

> Разница между моментами времени определяется неправильно,
> поскольку часовые пояса помечены не так, как нужно

In [35]:
#98 page
event1 = london_tz.localize(datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [36]:
event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

datetime.timedelta(days=1, seconds=18000)

Библиотека *pytz* включает списки наиболее распространенных часовых поясов и часовых поясов разных стран мира.

## Pyhon
Ознакомьтесь основными часовыми поясами *pytz.common_timezones*
> *pytz.common_timezones*
(длинный список...)

или часовыми поясами стран мира
> *pytz.country_timezones*('RU')

In [37]:
pytz.common_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

In [38]:
pytz.country_timezones('RU')

['Europe/Kaliningrad',
 'Europe/Moscow',
 'Europe/Kirov',
 'Europe/Volgograd',
 'Europe/Astrakhan',
 'Europe/Saratov',
 'Europe/Ulyanovsk',
 'Europe/Samara',
 'Asia/Yekaterinburg',
 'Asia/Omsk',
 'Asia/Novosibirsk',
 'Asia/Barnaul',
 'Asia/Tomsk',
 'Asia/Novokuznetsk',
 'Asia/Krasnoyarsk',
 'Asia/Irkutsk',
 'Asia/Chita',
 'Asia/Yakutsk',
 'Asia/Khandyga',
 'Asia/Vladivostok',
 'Asia/Ust-Nera',
 'Asia/Magadan',
 'Asia/Sakhalin',
 'Asia/Srednekolymsk',
 'Asia/Kamchatka',
 'Asia/Anadyr']

> *pytz.country_timezones('fr')*
> ('Europe/Paris'))

In [39]:
pytz.country_timezones('fr')

['Europe/Paris']

Особенно проблематичным является вопрос учета летнего времени. Некоторые используемые человечеством форматы времени учитываются компьютером дважды ( возврат к 'зимнему' времени осенью), тогда как другие вообще не учитываются (перевод времени вперед весной).

## Python
> Часовые пояса

In [40]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)

# Вывод результатов

In [41]:
#99 page
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [42]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

> # Последние две временные метки идентичны - это недопустимо!

>> В данном случае необходимо использовать параметр *is_dst*, чтобы
>> обозначить переход на летнее время

In [43]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [44]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)

In [45]:
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [46]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [47]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

## Повторяющиеся временные метки отсутствуют, по крайней мере до момента учета 

Трудности учета часового пояса могут не касаться решаемых вами задач, поэтому полезность приведенных выше сведений будет зависеть от характера обрабатываемых данных. Безусловно, существуют ситуации, в которых ошибка равнозначна катастрофе (скажем, экипажу дальномагистрального авиалайнера, который совершает рейс, проходящий точку перехода на летнее время, будет крайне неприятно осознавать, что в переданном прогнозе погоды такое событие никак не учитывается).
